# Jeju Hot Place

---
## 1. 게시글 정보 가져오기

In [1]:
import re

import time
import unicodedata

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [2]:
service=Service('../chromedriver/chromedriver.exe')
driver=webdriver.Chrome(service=service)

In [3]:
url='https://www.instagram.com/'
driver.get(url)

In [4]:
def insta_searching(word):
    url='https://www.instagram.com/explore/tags/{}/'.format(word)
    return (url)
word='제주도맛집'
url=insta_searching(word)
driver.get(url)
time.sleep(3)

In [5]:
#first=driver.find_element_by_css_selector('div._9AhH0')

first=driver.find_element(By.CSS_SELECTOR, 'div._9AhH0')
first.click()

def insta_searching(word):
    url='https://www.instagram.com/explore/tags/{}/'.format(word)
    return (url)

def select_first(word):
    first=driver.find_element(By.CSS_SELECTOR, 'div._9AhH0')
    first.click()
    time.sleep(5)

In [6]:
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

In [7]:
# 게시글 가져오기
content=soup.select('div.C4VMK>span')[0].text
content=unicodedata.normalize('NFC',content)
content

'#그날의기억방법 아기자기 귀여운 ‘세모 제주’동쪽 구좌읍 행원리에 주먹밥 테이크아웃점제주 재료들로 만든 음식이라서더 신선하고 맛있더라구요ㅎㅎㅎ외관도 너무 귀엽고 주먹밥도 귀여워서또 다시 생각나는 곳입니다☺️제주 여행에 출출할때 잠깐 들려서간단히 먹기 좋습니다!주말 잘 마무리하세요😌<제주 제주시 구좌읍 행원로 121>'

In [8]:
# 해시태그 가져오기
tags=re.findall(r'#[^\s#,\\]\w+',content)
tags

['#그날의기억방법']

In [9]:
# 작성일자 가져오기
date=soup.select('time')[0]['title']
date

'2022년 1월 16일'

In [10]:
# 좋아요 수 가져오기
like=soup.select('a.zV_Nj>span')[0].text
like

'4,138'

In [11]:
# 위치정보 가져오기
try:
    place=soup.select('div.JF9hh')[0].text
    place=unicodedata.normalize('NFC',place)
except:
    place=''
place

'세모제주'

In [12]:
def get_content(driver):
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    #본문내용
    try:
        content=soup.select('div.C4VMK>span')[0].text
        content=unicodedata.normalize('NFC',content)
    except:
        content=''
    #해시태그
    tags=re.findall(r'#[^\s#,\\]\w+',content)
    #작성일자
    date=soup.select('time')[0]['title']
    #좋아요
    like=soup.select('a.zV_Nj>span')[0].text
    #위치정보
    try:
        place=soup.select('div.JF9hh')[0].text
        place=unicodedata.normalize('NFC',place)
    except:
        place=''
    #수집한 정보 저장
    data=[content,date,like,place,tags]
    return data

In [13]:
get_content(driver)

['#그날의기억방법 아기자기 귀여운 ‘세모 제주’동쪽 구좌읍 행원리에 주먹밥 테이크아웃점제주 재료들로 만든 음식이라서더 신선하고 맛있더라구요ㅎㅎㅎ외관도 너무 귀엽고 주먹밥도 귀여워서또 다시 생각나는 곳입니다☺️제주 여행에 출출할때 잠깐 들려서간단히 먹기 좋습니다!주말 잘 마무리하세요😌<제주 제주시 구좌읍 행원로 121>',
 '2022년 1월 16일',
 '4,138',
 '세모제주',
 ['#그날의기억방법']]

---
## 2. 여러 게시글 정보 수집하기

In [14]:
#필요한 함수
def insta_searching(word):
    url='https://www.instagram.com/explore/tags/{}/'.format(word)
    return (url)

def select_first(word):
    first=driver.find_element(By.CSS_SELECTOR, 'div._9AhH0')
    first.click()
    time.sleep(5)
    
# 다음 컨텐츠로 넘어가기
def move_next(driver):
    next_c=driver.find_element(By.CSS_SELECTOR, 'div.l8mY4.feth3')
    next_c.click()
    time.sleep(3)
    
def get_content(driver):
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    
    #본문내용
    content=soup.select('div.C4VMK>span')[0].text
    content=unicodedata.normalize('NFC',content)

    #해시태그
    tags=re.findall(r'#[^\s#,\\]\w+',content)
    #작성일자
    date=soup.select('time')[0]['title']
    #좋아요
    like=soup.select('a.zV_Nj>span')[0].text
    #위치정보
    try:
        place=soup.select('div.JF9hh')[0].text
        place=unicodedata.normalize('NFC',place)
    except:
        place=''
    #수집한 정보 저장
    data=[content,date,like,place,tags]
    return data

In [15]:
service=Service('../chromedriver/chromedriver.exe')
driver=webdriver.Chrome(service=service)
time.sleep(2)
url='https://www.instagram.com/'
driver.get(url)

In [16]:
def insta_crawling(word,n):
    #url 띄우기
    url=insta_searching(word)
    driver.get(url)
    time.sleep(5)
    #클릭하기
    select_first(driver)
    
    res=[]
    
    for i in range(n):
        try:
            data=get_content(driver)
            res.append(data)
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)
    
    return(res)

In [17]:
res111=insta_crawling('제주도맛집',20)

In [18]:
len(res111)

9

---
# 3. 수집 데이터 저장

In [19]:
res_df=pd.DataFrame(res111)
res_df.columns=['content','date','like','place','tags']
res_df.to_excel('./files/crawling_jj.xlsx',index=False)

In [20]:
res_df.head()

,content,date,like,place,tags
0,#그날의기억방법 아기자기 귀여운 ‘세모 제주’동쪽 구좌읍 행원리에 주먹밥 테이크아웃...,2022년 1월 16일,"4,140",세모제주,[#그날의기억방법]
1,(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통...,2022년 1월 15일,295,,"[#연남동맛집, #제주도맛집, #중문맛집, #제주공항맛집, #제주공항근처맛집, #서..."
2,#그날의기억방법효리네민박에서 신상 소품샵으로 ‘소길별하’애월에 생긴 요즘 핫하다는 ...,2022년 1월 17일,"2,812",소길별하,[#그날의기억방법효리네민박에서]
3,제주 산방산 가기전 먹방투어출출할 때 유명한 #천지연숯화로구이 에서!!흑돼지오겹살 ...,2022년 1월 17일,"1,597",,"[#천지연숯화로구이, #천지연숯화로구이, #산방산맛집, #송악산맛집, #산방산흑돼지..."
4,📍제주도_표선✅ #메밀밭에가시리 / 메밀냉면하트점수 : ♥️♥️♥️♥️♥️🖤🖤⠀<제...,2022년 1월 15일,"1,554",메밀밭에가시리,"[#메밀밭에가시리, #메밀밭에가시리, #메밀밭에가시리, #메밀밭에가시리, #함스타_..."


---
# 4. 여러 데이터의 중복을 제거하여 통합저장하기

In [21]:
jeju_insta_df=pd.DataFrame( [] )
folder='./files/'
f_list=['1_crawling_jejudoGwanGwang.xlsx',
       '1_crawling_jejudoMatJip.xlsx',
       '1_crawling_jejuMatJip.xlsx',
       '1_crawling_jejuYeoHang.xlsx']

for fname in f_list:
    f_pth=folder+fname
    tmp=pd.read_excel(f_pth)
    jeju_insta_df=jeju_insta_df.append(tmp)
    
jeju_insta_df.columns=['content','date','like','place','tags']

In [22]:
jeju_insta_df.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [23]:
print(len(jeju_insta_df))
jeju_insta_df.drop_duplicates(subset=['content'],inplace=True)
print(len(jeju_insta_df))
jeju_insta_df.to_excel('./files/1_crawling_raw.xlsx',index=False)

12780
8369
